In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import dicomsdl
import gdcm
from tqdm import tqdm
import glob

In [6]:

DATA_DIR = '../../data'
DATASET_DIR = os.path.join(DATA_DIR, 'dataset')
data_type = 'train'
test_dir = os.path.join(DATASET_DIR, f'{data_type}_images')

df = pd.read_csv(os.path.join(DATASET_DIR, f'{data_type}_series_meta.csv'))
df['psid'] = df['patient_id'].astype(str) + '_' + df['series_id'].astype(str)
df['image_folder'] = test_dir + '/' + df['patient_id'].astype(str) + '/' + df['series_id'].astype(str)
df

,patient_id,series_id,aortic_hu,incomplete_organ,psid,image_folder
0,10004,21057,146.00,0,10004_21057,../../data/dataset/train_images/10004/21057
1,10004,51033,454.75,0,10004_51033,../../data/dataset/train_images/10004/51033
2,10005,18667,187.00,0,10005_18667,../../data/dataset/train_images/10005/18667
3,10007,47578,329.00,0,10007_47578,../../data/dataset/train_images/10007/47578
4,10026,29700,327.00,0,10026_29700,../../data/dataset/train_images/10026/29700
...,...,...,...,...,...,...
4706,9961,2003,381.00,0,9961_2003,../../data/dataset/train_images/9961/2003
4707,9961,63032,143.75,0,9961_63032,../../data/dataset/train_images/9961/63032
4708,9980,40214,103.00,0,9980_40214,../../data/dataset/train_images/9980/40214
4709,9980,40466,135.00,0,9980_40466,../../data/dataset/train_images/9980/40466


In [ ]:
def dicomsdl_to_numpy_image(ds, index=0):
    info = ds.getPixelDataInfo()
    if info['SamplesPerPixel'] != 1:
        raise RuntimeError('SamplesPerPixel != 1')  # number of separate planes in this image
    shape = [info['Rows'], info['Cols']]
    dtype = info['dtype']
    outarr = np.empty(shape, dtype=dtype)
    ds.copyFrameData(index, outarr)
    return outarr

In [ ]:
def check_inverse(t_paths):
    n_scans = len(t_paths)
    
    #check inversion
    min_index, max_index = t_paths[0], t_paths[-1]
    dcm0 = dicomsdl.open(min_index)
    dcmN = dicomsdl.open(max_index)
    sx0, sy0, sz0 = dcm0.ImagePositionPatient
    sxN, syN, szN = dcmN.ImagePositionPatient
    
    inversion = True if szN < sz0 else False
    if inversion:
        t_paths = t_paths[::-1]
        print()
    return t_paths, inversion

d2 = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    psid = str(row['psid'])
    pid, sid = str(row['patient_id']), str(row['series_id'])
    t_paths = sorted(glob.glob(os.path.join(test_dir, pid, sid, "*")), key=lambda x: int(x.split('/')[-1].split(".")[0]))    
    t_paths, inverse = check_inverse(t_paths)
    d2[psid] = inverse

In [ ]:
df2 = pd.DataFrame.from_dict(d2, orient='index', columns=['inverse']).reset_index().rename(columns={'index':'psid'})
df2

In [ ]:
df2.to_csv(os.path.join(DATA_DIR, "inverse.csv"), index=False)